# question 2 | UCI

In [1]:
# import moduls 
import numpy as np
import pandas as pd
# sklearn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

# import collections
from collections import Counter

In [2]:
Main_df= pd.read_csv("diabetes+130-us+hospitals+for+years+1999-2008/diabetic_data.csv")
Main_ids_mapping = pd.read_csv("diabetes+130-us+hospitals+for+years+1999-2008/ids_mapping.csv")
diabetic_df = Main_df
ids_mapping = Main_ids_mapping
display(diabetic_df.head())
display(diabetic_df.info())

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

None

# handel Identifiers columns

In [3]:
display('len == number of rows',len(Counter(diabetic_df["encounter_id"])))
display('len != number of rows',len(Counter(diabetic_df["patient_nbr"])))



'len == number of rows'

101766

'len != number of rows'

71518

فاقد ارزش محاسباتی راجع به شرایط بیمار هستند پس بهتره حذف بشن 

# handel race column

In [4]:
from sklearn.preprocessing import OneHotEncoder

diabetic_df['race'] = diabetic_df['race'].replace('?', 'Other')

one_hot_encoder = OneHotEncoder(
    handle_unknown='ignore',
)

encoded = one_hot_encoder.fit_transform(diabetic_df[['race']])

encoded_df = pd.DataFrame(
    encoded.toarray(),
    columns=one_hot_encoder.get_feature_names_out(['race']),
    index=diabetic_df.index
)

diabetic_df = pd.concat([diabetic_df.drop(columns=['race']), encoded_df], axis=1)


display(diabetic_df.head())


,encounter_id,patient_nbr,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,...,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other
0,2278392,8222157,Female,[0-10),?,6,25,1,1,?,...,No,No,No,No,NO,0.0,0.0,1.0,0.0,0.0
1,149190,55629189,Female,[10-20),?,1,1,7,3,?,...,No,No,Ch,Yes,>30,0.0,0.0,1.0,0.0,0.0
2,64410,86047875,Female,[20-30),?,1,1,7,2,?,...,No,No,No,Yes,NO,1.0,0.0,0.0,0.0,0.0
3,500364,82442376,Male,[30-40),?,1,1,7,2,?,...,No,No,Ch,Yes,NO,0.0,0.0,1.0,0.0,0.0
4,16680,42519267,Male,[40-50),?,1,1,7,1,?,...,No,No,Ch,Yes,NO,0.0,0.0,1.0,0.0,0.0


# handel weight column

In [5]:
display(Counter(diabetic_df['weight']))

diabetic_df['weight_missing'] = (diabetic_df['weight'] == '?').astype(int)

diabetic_df['weight_num'] =diabetic_df['weight'].map({'[75-100)':np.mean([75,100]),'[50-75)':np.mean([50,75]),'[100-125)'\
:np.mean([100,125]),'[125-150)':np.mean([125,150]),'[25-50)':np.mean([25,50]),'[0-25)':np.mean([0,25])\
,'[150-175)':np.mean([150,175]),'[175-200)':np.mean([175,200]),'>200':200})

diabetic_df['weight_num'] = diabetic_df['weight_num'].fillna(0)

diabetic_df.drop(columns=['weight'],inplace=True)

display(diabetic_df.head())


Counter({'?': 98569,
         '[75-100)': 1336,
         '[50-75)': 897,
         '[100-125)': 625,
         '[125-150)': 145,
         '[25-50)': 97,
         '[0-25)': 48,
         '[150-175)': 35,
         '[175-200)': 11,
         '>200': 3})

,encounter_id,patient_nbr,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,...,change,diabetesMed,readmitted,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other,weight_missing,weight_num
0,2278392,8222157,Female,[0-10),6,25,1,1,?,Pediatrics-Endocrinology,...,No,No,NO,0.0,0.0,1.0,0.0,0.0,1,0.0
1,149190,55629189,Female,[10-20),1,1,7,3,?,?,...,Ch,Yes,>30,0.0,0.0,1.0,0.0,0.0,1,0.0
2,64410,86047875,Female,[20-30),1,1,7,2,?,?,...,No,Yes,NO,1.0,0.0,0.0,0.0,0.0,1,0.0
3,500364,82442376,Male,[30-40),1,1,7,2,?,?,...,Ch,Yes,NO,0.0,0.0,1.0,0.0,0.0,1,0.0
4,16680,42519267,Male,[40-50),1,1,7,1,?,?,...,Ch,Yes,NO,0.0,0.0,1.0,0.0,0.0,1,0.0


# handel gender column

In [6]:
display(Counter(diabetic_df['gender']))
diabetic_df['gender'] = diabetic_df['gender'].map({'Unknown/Invalid':'Male','Female':'Female','Male':'Male'})
print('map all Unknown to Male')
display(Counter(diabetic_df["gender"]))
# one hot encoding

one_hot_encoder = OneHotEncoder(
    handle_unknown='ignore',
)

encoded = one_hot_encoder.fit_transform(diabetic_df[['gender']])

encoded_df = pd.DataFrame(
    encoded.toarray(),
    columns=one_hot_encoder.get_feature_names_out(['gender']),
    index=diabetic_df.index
)

diabetic_df = pd.concat([diabetic_df.drop(columns=['gender']), encoded_df], axis=1)

display(diabetic_df.head())

Counter({'Female': 54708, 'Male': 47055, 'Unknown/Invalid': 3})

map all Unknown to Male


Counter({'Female': 54708, 'Male': 47058})

,encounter_id,patient_nbr,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,readmitted,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other,weight_missing,weight_num,gender_Female,gender_Male
0,2278392,8222157,[0-10),6,25,1,1,?,Pediatrics-Endocrinology,41,...,NO,0.0,0.0,1.0,0.0,0.0,1,0.0,1.0,0.0
1,149190,55629189,[10-20),1,1,7,3,?,?,59,...,>30,0.0,0.0,1.0,0.0,0.0,1,0.0,1.0,0.0
2,64410,86047875,[20-30),1,1,7,2,?,?,11,...,NO,1.0,0.0,0.0,0.0,0.0,1,0.0,1.0,0.0
3,500364,82442376,[30-40),1,1,7,2,?,?,44,...,NO,0.0,0.0,1.0,0.0,0.0,1,0.0,0.0,1.0
4,16680,42519267,[40-50),1,1,7,1,?,?,51,...,NO,0.0,0.0,1.0,0.0,0.0,1,0.0,0.0,1.0


# handel age column

In [7]:
display(Counter(diabetic_df['age']))
diabetic_df['age'] = diabetic_df['age'].map({'[0-10)':np.mean([0,10]),'[10-20)':np.mean([10,20]),\
                                             '[20-30)':np.mean([20,30]),'[30-40)':np.mean([30,40]),\
                                            '[40-50)':np.mean([40,50]),'[50-60)': np.mean([50,60]),'[60-70)':np.mean([60,70]) ,\
                                             '[70-80)': np.mean([70,80]),'[80-90)':np.mean([80,90]),'[90-100)':np.mean([90,100])})
print('range to mean changed successfully')
display(Counter(diabetic_df["age"]))
display(diabetic_df.head())

Counter({'[70-80)': 26068,
         '[60-70)': 22483,
         '[50-60)': 17256,
         '[80-90)': 17197,
         '[40-50)': 9685,
         '[30-40)': 3775,
         '[90-100)': 2793,
         '[20-30)': 1657,
         '[10-20)': 691,
         '[0-10)': 161})

range to mean changed successfully


Counter({75.0: 26068,
         65.0: 22483,
         55.0: 17256,
         85.0: 17197,
         45.0: 9685,
         35.0: 3775,
         95.0: 2793,
         25.0: 1657,
         15.0: 691,
         5.0: 161})

,encounter_id,patient_nbr,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,readmitted,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other,weight_missing,weight_num,gender_Female,gender_Male
0,2278392,8222157,5.0,6,25,1,1,?,Pediatrics-Endocrinology,41,...,NO,0.0,0.0,1.0,0.0,0.0,1,0.0,1.0,0.0
1,149190,55629189,15.0,1,1,7,3,?,?,59,...,>30,0.0,0.0,1.0,0.0,0.0,1,0.0,1.0,0.0
2,64410,86047875,25.0,1,1,7,2,?,?,11,...,NO,1.0,0.0,0.0,0.0,0.0,1,0.0,1.0,0.0
3,500364,82442376,35.0,1,1,7,2,?,?,44,...,NO,0.0,0.0,1.0,0.0,0.0,1,0.0,0.0,1.0
4,16680,42519267,45.0,1,1,7,1,?,?,51,...,NO,0.0,0.0,1.0,0.0,0.0,1,0.0,0.0,1.0


# handel admission type id column

In [8]:
admission_map = {
    1: 'Emergency',
    2: 'Urgent',
    3: 'Elective',
    4: 'Newborn',
    5: 'Other',
    6: 'Other',
    7: 'Trauma',
    8: 'Other'
}

diabetic_df['admission_type'] = diabetic_df['admission_type_id'].map(admission_map)

# one-hot
admission_ohe = pd.get_dummies(diabetic_df['admission_type'], prefix='admission')

diabetic_df = pd.concat([diabetic_df, admission_ohe], axis=1)
diabetic_df.drop(columns=['admission_type_id', 'admission_type'], inplace=True)

display(diabetic_df.head())

,encounter_id,patient_nbr,age,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,...,weight_missing,weight_num,gender_Female,gender_Male,admission_Elective,admission_Emergency,admission_Newborn,admission_Other,admission_Trauma,admission_Urgent
0,2278392,8222157,5.0,25,1,1,?,Pediatrics-Endocrinology,41,0,...,1,0.0,1.0,0.0,False,False,False,True,False,False
1,149190,55629189,15.0,1,7,3,?,?,59,0,...,1,0.0,1.0,0.0,False,True,False,False,False,False
2,64410,86047875,25.0,1,7,2,?,?,11,5,...,1,0.0,1.0,0.0,False,True,False,False,False,False
3,500364,82442376,35.0,1,7,2,?,?,44,1,...,1,0.0,0.0,1.0,False,True,False,False,False,False
4,16680,42519267,45.0,1,7,1,?,?,51,0,...,1,0.0,0.0,1.0,False,True,False,False,False,False


# handel discharge disposition id 

In [9]:
discharge_map = {
    1: 'Home',
    6: 'Home',
    8: 'Home',

    2: 'Facility',
    3: 'Facility',
    4: 'Facility',

    5: 'Hospice',
    18: 'Hospice',

    7: 'Left_AMA',
    11: 'Expired'
}

diabetic_df['discharge_group'] = diabetic_df['discharge_disposition_id'].map(discharge_map)

discharge_ohe = pd.get_dummies(
    diabetic_df['discharge_group'],
    prefix='discharge'
)

diabetic_df = pd.concat([diabetic_df, discharge_ohe], axis=1)
diabetic_df.drop(columns=['discharge_disposition_id', 'discharge_group'], inplace=True)

display(diabetic_df.head())

,encounter_id,patient_nbr,age,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,...,admission_Emergency,admission_Newborn,admission_Other,admission_Trauma,admission_Urgent,discharge_Expired,discharge_Facility,discharge_Home,discharge_Hospice,discharge_Left_AMA
0,2278392,8222157,5.0,1,1,?,Pediatrics-Endocrinology,41,0,1,...,False,False,True,False,False,False,False,False,False,False
1,149190,55629189,15.0,7,3,?,?,59,0,18,...,True,False,False,False,False,False,False,True,False,False
2,64410,86047875,25.0,7,2,?,?,11,5,13,...,True,False,False,False,False,False,False,True,False,False
3,500364,82442376,35.0,7,2,?,?,44,1,16,...,True,False,False,False,False,False,False,True,False,False
4,16680,42519267,45.0,7,1,?,?,51,0,8,...,True,False,False,False,False,False,False,True,False,False


# handel admission_source_id

In [10]:
source_map = {
    7: 'Emergency',

    1: 'Referral',
    2: 'Referral',
    3: 'Referral',

    17: 'Transfer',
    18: 'Transfer',

    4: 'Other',
    5: 'Other',
    6: 'Other',
    8: 'Other',
    9: 'Other',
    10: 'Other',
    22: 'Other'
}

diabetic_df['admission_source'] = diabetic_df['admission_source_id'].map(source_map)

source_ohe = pd.get_dummies(
    diabetic_df['admission_source'],
    prefix='source'
)

diabetic_df = pd.concat([diabetic_df, source_ohe], axis=1)
diabetic_df.drop(columns=['admission_source_id', 'admission_source'], inplace=True)

display(diabetic_df.head())

,encounter_id,patient_nbr,age,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,admission_Urgent,discharge_Expired,discharge_Facility,discharge_Home,discharge_Hospice,discharge_Left_AMA,source_Emergency,source_Other,source_Referral,source_Transfer
0,2278392,8222157,5.0,1,?,Pediatrics-Endocrinology,41,0,1,0,...,False,False,False,False,False,False,False,False,True,False
1,149190,55629189,15.0,3,?,?,59,0,18,0,...,False,False,False,True,False,False,True,False,False,False
2,64410,86047875,25.0,2,?,?,11,5,13,2,...,False,False,False,True,False,False,True,False,False,False
3,500364,82442376,35.0,2,?,?,44,1,16,0,...,False,False,False,True,False,False,True,False,False,False
4,16680,42519267,45.0,1,?,?,51,0,8,0,...,False,False,False,True,False,False,True,False,False,False


به جهت حفظ تفسیر پذیری و به جهت اینکه رندوم فورست و اکس جی بوست میتوانند با تعیین آستانه این شناسه ها رو تفکیک کنن و از طرف دیگر تعداد ستون ها در صورت وان هات انکودینگ بسیار زیادی میشود این فیچر هم نیاز به پردازش ندارد 

# handel time_in_hospital column

In [11]:
display(Counter(diabetic_df['time_in_hospital']))
display(diabetic_df.head())

Counter({3: 17756,
         2: 17224,
         1: 14208,
         4: 13924,
         5: 9966,
         6: 7539,
         7: 5859,
         8: 4391,
         9: 3002,
         10: 2342,
         11: 1855,
         12: 1448,
         13: 1210,
         14: 1042})

,encounter_id,patient_nbr,age,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,admission_Urgent,discharge_Expired,discharge_Facility,discharge_Home,discharge_Hospice,discharge_Left_AMA,source_Emergency,source_Other,source_Referral,source_Transfer
0,2278392,8222157,5.0,1,?,Pediatrics-Endocrinology,41,0,1,0,...,False,False,False,False,False,False,False,False,True,False
1,149190,55629189,15.0,3,?,?,59,0,18,0,...,False,False,False,True,False,False,True,False,False,False
2,64410,86047875,25.0,2,?,?,11,5,13,2,...,False,False,False,True,False,False,True,False,False,False
3,500364,82442376,35.0,2,?,?,44,1,16,0,...,False,False,False,True,False,False,True,False,False,False
4,16680,42519267,45.0,1,?,?,51,0,8,0,...,False,False,False,True,False,False,True,False,False,False


این فیچر نباید انکود بشه چون  تعداد روز های بستری شدن فرد هست پس میتونه با بالا رفتن شدت بیماری رو مشخص کنه 

نیازی به اسکیل شدن هم نداره چون رندوم فورست و اکس جی بوست نیازی به اسکیل شدن ندارن 

# handel payer_code column

In [12]:
display(Counter(diabetic_df['payer_code']))
diabetic_df.drop(columns=['payer_code'], inplace=True)

display(diabetic_df.head())

Counter({'?': 40256,
         'MC': 32439,
         'HM': 6274,
         'SP': 5007,
         'BC': 4655,
         'MD': 3532,
         'CP': 2533,
         'UN': 2448,
         'CM': 1937,
         'OG': 1033,
         'PO': 592,
         'DM': 549,
         'CH': 146,
         'WC': 135,
         'OT': 95,
         'MP': 79,
         'SI': 55,
         'FR': 1})

,encounter_id,patient_nbr,age,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,...,admission_Urgent,discharge_Expired,discharge_Facility,discharge_Home,discharge_Hospice,discharge_Left_AMA,source_Emergency,source_Other,source_Referral,source_Transfer
0,2278392,8222157,5.0,1,Pediatrics-Endocrinology,41,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
1,149190,55629189,15.0,3,?,59,0,18,0,0,...,False,False,False,True,False,False,True,False,False,False
2,64410,86047875,25.0,2,?,11,5,13,2,0,...,False,False,False,True,False,False,True,False,False,False
3,500364,82442376,35.0,2,?,44,1,16,0,0,...,False,False,False,True,False,False,True,False,False,False
4,16680,42519267,45.0,1,?,51,0,8,0,0,...,False,False,False,True,False,False,True,False,False,False


# handel medical_specialty column

In [13]:
display(Counter(diabetic_df['medical_specialty']))

diabetic_df.drop(columns=['medical_specialty'], inplace=True)

display(diabetic_df.head())


Counter({'?': 49949,
         'InternalMedicine': 14635,
         'Emergency/Trauma': 7565,
         'Family/GeneralPractice': 7440,
         'Cardiology': 5352,
         'Surgery-General': 3099,
         'Nephrology': 1613,
         'Orthopedics': 1400,
         'Orthopedics-Reconstructive': 1233,
         'Radiologist': 1140,
         'Pulmonology': 871,
         'Psychiatry': 854,
         'Urology': 685,
         'ObstetricsandGynecology': 671,
         'Surgery-Cardiovascular/Thoracic': 652,
         'Gastroenterology': 564,
         'Surgery-Vascular': 533,
         'Surgery-Neuro': 468,
         'PhysicalMedicineandRehabilitation': 391,
         'Oncology': 348,
         'Pediatrics': 254,
         'Hematology/Oncology': 207,
         'Neurology': 203,
         'Pediatrics-Endocrinology': 159,
         'Otolaryngology': 125,
         'Endocrinology': 120,
         'Surgery-Thoracic': 109,
         'Psychology': 101,
         'Podiatry': 100,
         'Surgery-Cardiovascular': 98

,encounter_id,patient_nbr,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,admission_Urgent,discharge_Expired,discharge_Facility,discharge_Home,discharge_Hospice,discharge_Left_AMA,source_Emergency,source_Other,source_Referral,source_Transfer
0,2278392,8222157,5.0,1,41,0,1,0,0,0,...,False,False,False,False,False,False,False,False,True,False
1,149190,55629189,15.0,3,59,0,18,0,0,0,...,False,False,False,True,False,False,True,False,False,False
2,64410,86047875,25.0,2,11,5,13,2,0,1,...,False,False,False,True,False,False,True,False,False,False
3,500364,82442376,35.0,2,44,1,16,0,0,0,...,False,False,False,True,False,False,True,False,False,False
4,16680,42519267,45.0,1,51,0,8,0,0,0,...,False,False,False,True,False,False,True,False,False,False


# handel num_lab_procedures column

In [14]:
print('sum of nulls in num_lab_procedures = ',np.sum(diabetic_df['num_lab_procedures'].isnull()))
print('sum of nulls in num_procedures = ',np.sum(diabetic_df['num_procedures'].isnull()))
print('sum of nulls in num_medications = ',np.sum(diabetic_df['num_medications'].isnull()))
print('sum of nulls in num_outpatient = ',np.sum(diabetic_df['number_outpatient'].isnull()))
print('sum of nulls in num_emergency = ',np.sum(diabetic_df['number_emergency'].isnull()))
print('sum of nulls in num_inpatient = ',np.sum(diabetic_df['number_inpatient'].isnull()))

display(diabetic_df[['num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency','number_inpatient']].info())

sum of nulls in num_lab_procedures =  0
sum of nulls in num_procedures =  0
sum of nulls in num_medications =  0
sum of nulls in num_outpatient =  0
sum of nulls in num_emergency =  0
sum of nulls in num_inpatient =  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype
---  ------              --------------   -----
 0   num_lab_procedures  101766 non-null  int64
 1   num_procedures      101766 non-null  int64
 2   num_medications     101766 non-null  int64
 3   number_outpatient   101766 non-null  int64
 4   number_emergency    101766 non-null  int64
 5   number_inpatient    101766 non-null  int64
dtypes: int64(6)
memory usage: 4.7 MB


None

هیچ کدوم نیازی به انکود ندارن و نشان از تعداد دفعات اعمال انجام شده و... برای بیمار دارن که وخیم بودن وضعیت رو میتونه برسونه 


i

# handel diag_1...3

In [15]:
def icd9_to_group(code):
    try:
        code = str(code)
        if code.startswith('V') or code.startswith('E'):
            return 'Other'
        code_num = float(code)

        if 390 <= code_num <= 459 or code_num == 785:
            return 'Circulatory'
        elif 460 <= code_num <= 519:
            return 'Respiratory'
        elif 520 <= code_num <= 579:
            return 'Digestive'
        elif code_num == 250:
            return 'Diabetes'
        elif 800 <= code_num <= 999:
            return 'Injury'
        elif 710 <= code_num <= 739:
            return 'Musculoskeletal'
        elif 580 <= code_num <= 629:
            return 'Genitourinary'
        elif 140 <= code_num <= 239:
            return 'Neoplasms'
        else:
            return 'Other'
    except:
        return 'Other'

for col in ['diag_1', 'diag_2', 'diag_3']:
    diabetic_df[col + '_group'] = diabetic_df[col].apply(icd9_to_group)

diag_ohe = pd.get_dummies(
    diabetic_df[['diag_1_group', 'diag_2_group', 'diag_3_group']],
    prefix=['diag1', 'diag2', 'diag3']
)

diabetic_df = pd.concat([diabetic_df, diag_ohe], axis=1)
diabetic_df.drop(columns=['diag_1','diag_2','diag_3',
                           'diag_1_group','diag_2_group','diag_3_group'],
                 inplace=True)

display(diabetic_df.head())

,encounter_id,patient_nbr,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,diag2_Respiratory,diag3_Circulatory,diag3_Diabetes,diag3_Digestive,diag3_Genitourinary,diag3_Injury,diag3_Musculoskeletal,diag3_Neoplasms,diag3_Other,diag3_Respiratory
0,2278392,8222157,5.0,1,41,0,1,0,0,0,...,False,False,False,False,False,False,False,False,True,False
1,149190,55629189,15.0,3,59,0,18,0,0,0,...,False,False,False,False,False,False,False,False,True,False
2,64410,86047875,25.0,2,11,5,13,2,0,1,...,False,False,False,False,False,False,False,False,True,False
3,500364,82442376,35.0,2,44,1,16,0,0,0,...,False,True,False,False,False,False,False,False,False,False
4,16680,42519267,45.0,1,51,0,8,0,0,0,...,False,False,True,False,False,False,False,False,False,False


# handel number_diagnoses

اینم نیازی به انکود نداره 

In [16]:
print('sum of nulls in number_diagnoses = ',np.sum(diabetic_df['number_diagnoses'].isnull()))
display(diabetic_df[['number_diagnoses']].info())


sum of nulls in number_diagnoses =  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 1 columns):
 #   Column            Non-Null Count   Dtype
---  ------            --------------   -----
 0   number_diagnoses  101766 non-null  int64
dtypes: int64(1)
memory usage: 795.2 KB


None

# handel max_glu_serum

In [17]:
display(Counter(diabetic_df['max_glu_serum']))

diabetic_df['max_glu_measured'] = diabetic_df['max_glu_serum'].notna().astype(int)
diabetic_df['max_glu_high'] = diabetic_df['max_glu_serum'].isin(['>200', '>300']).astype(int)
diabetic_df.drop(columns=['max_glu_serum'], inplace=True)

display(diabetic_df.head())

Counter({nan: 96420, 'Norm': 2597, '>200': 1485, '>300': 1264})

,encounter_id,patient_nbr,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,diag3_Diabetes,diag3_Digestive,diag3_Genitourinary,diag3_Injury,diag3_Musculoskeletal,diag3_Neoplasms,diag3_Other,diag3_Respiratory,max_glu_measured,max_glu_high
0,2278392,8222157,5.0,1,41,0,1,0,0,0,...,False,False,False,False,False,False,True,False,0,0
1,149190,55629189,15.0,3,59,0,18,0,0,0,...,False,False,False,False,False,False,True,False,0,0
2,64410,86047875,25.0,2,11,5,13,2,0,1,...,False,False,False,False,False,False,True,False,0,0
3,500364,82442376,35.0,2,44,1,16,0,0,0,...,False,False,False,False,False,False,False,False,0,0
4,16680,42519267,45.0,1,51,0,8,0,0,0,...,True,False,False,False,False,False,False,False,0,0


# handel A1C result

In [18]:
display(Counter(diabetic_df['A1Cresult']))

diabetic_df['a1c_measured'] = diabetic_df['A1Cresult'].notna().astype(int)
diabetic_df['a1c_high'] = diabetic_df['A1Cresult'].isin(['>7', '>8']).astype(int)
diabetic_df.drop(columns=['A1Cresult'], inplace=True)


display(diabetic_df.head())

Counter({nan: 84748, '>8': 8216, 'Norm': 4990, '>7': 3812})

,encounter_id,patient_nbr,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,diag3_Genitourinary,diag3_Injury,diag3_Musculoskeletal,diag3_Neoplasms,diag3_Other,diag3_Respiratory,max_glu_measured,max_glu_high,a1c_measured,a1c_high
0,2278392,8222157,5.0,1,41,0,1,0,0,0,...,False,False,False,False,True,False,0,0,0,0
1,149190,55629189,15.0,3,59,0,18,0,0,0,...,False,False,False,False,True,False,0,0,0,0
2,64410,86047875,25.0,2,11,5,13,2,0,1,...,False,False,False,False,True,False,0,0,0,0
3,500364,82442376,35.0,2,44,1,16,0,0,0,...,False,False,False,False,False,False,0,0,0,0
4,16680,42519267,45.0,1,51,0,8,0,0,0,...,False,False,False,False,False,False,0,0,0,0


# handel No / Steady / Up / Down faetures

In [19]:

med_cols = [
    'metformin','repaglinide','nateglinide','chlorpropamide',
    'glimepiride','glyburide','acetohexamide','glipizide',
    'tolbutamide','pioglitazone','rosiglitazone','acarbose',
    'miglitol','troglitazone','tolazamide','examide','citoglipton'
]

for col in med_cols:
    diabetic_df[col + '_used'] = (diabetic_df[col] != 'No').astype(int)

diabetic_df['insulin_used'] = (diabetic_df['insulin'] != 'No').astype(int)

diabetic_df['med_changed'] = (diabetic_df['change'] == 'Ch').astype(int)

diabetic_df.drop(columns=med_cols + ['insulin', 'change'], inplace=True)

combo_cols = [
    'glyburide-metformin', 'glipizide-metformin',
    'glimepiride-pioglitazone', 'metformin-rosiglitazone',
    'metformin-pioglitazone'
]
diabetic_df.drop(columns=combo_cols, inplace=True)

diabetic_df.filter(like='_used').head()



diabetic_df['readmitted'] = (diabetic_df['readmitted'] == '<30').astype(int)

diabetic_df['readmitted'].value_counts()


display(diabetic_df.head())


,encounter_id,patient_nbr,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,pioglitazone_used,rosiglitazone_used,acarbose_used,miglitol_used,troglitazone_used,tolazamide_used,examide_used,citoglipton_used,insulin_used,med_changed
0,2278392,8222157,5.0,1,41,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,149190,55629189,15.0,3,59,0,18,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,64410,86047875,25.0,2,11,5,13,2,0,1,...,0,0,0,0,0,0,0,0,0,0
3,500364,82442376,35.0,2,44,1,16,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,16680,42519267,45.0,1,51,0,8,0,0,0,...,0,0,0,0,0,0,0,0,1,1


# handel 'diabetesMed'

In [20]:
# ---- Fix remaining object column (diabetesMed) ----

print("Before encoding diabetesMed:")
print(Counter(diabetic_df['diabetesMed']))

# binary encoding (Yes / No)
diabetic_df['diabetesMed'] = diabetic_df['diabetesMed'].map({
    'Yes': 1,
    'No': 0
})

print("After encoding diabetesMed:")
print(Counter(diabetic_df['diabetesMed']))

# sanity check: no object columns should remain
obj_cols = diabetic_df.select_dtypes(include='object').columns
print("Remaining object columns:", obj_cols)


Before encoding diabetesMed:
Counter({'Yes': 78363, 'No': 23403})
After encoding diabetesMed:
Counter({1: 78363, 0: 23403})
Remaining object columns: Index([], dtype='object')


# save numerical dataset to CSV  for part 2 of this question

In [21]:
diabetic_df.to_csv('diabetes+130-us+hospitals+for+years+1999-2008/encoded.csv',index=False)